In [1]:
import pandas as pd
import numpy as np
import os, sys
import glob

In [2]:
os.remove('./generation_forecast/.DS_Store')


FileNotFoundError: [Errno 2] No such file or directory: './generation_forecast/.DS_Store'

In [113]:
import glob

solaire_files = glob.glob('./generation_forecast/*Solaire*.csv')
eolien_files = glob.glob('./generation_forecast/*Eolien*.csv')
print(solaire_files)

['./generation_forecast/PrevisionSolaire_2020.xls.csv', './generation_forecast/PrevisionSolaire_2021.xls.csv', './generation_forecast/PrevisionSolaire_2017.xls.csv', './generation_forecast/PrevisionSolaire_2023.xls.csv', './generation_forecast/PrevisionSolaire_2019.xls.csv', './generation_forecast/PrevisionSolaire_2018.xls.csv', './generation_forecast/PrevisionSolaire_2022.xls.csv']


In [127]:
files = [solaire_files, eolien_files]
df_list = []
for i, file in enumerate(solaire_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list.append(df)


In [129]:
solaire_files

['./generation_forecast/PrevisionSolaire_2020.xls.csv',
 './generation_forecast/PrevisionSolaire_2021.xls.csv',
 './generation_forecast/PrevisionSolaire_2017.xls.csv',
 './generation_forecast/PrevisionSolaire_2023.xls.csv',
 './generation_forecast/PrevisionSolaire_2019.xls.csv',
 './generation_forecast/PrevisionSolaire_2018.xls.csv',
 './generation_forecast/PrevisionSolaire_2022.xls.csv']

In [159]:

df_solaire = pd.concat(df_list)

#df_solaire = df_solaire.reset_index(inplace=False).drop('index', axis=1)

mask = df_solaire['Heure'].isna()
df_solaire=df_solaire.loc[~mask]

df_solaire['Heure'] = df_solaire['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_solaire['Date_Heure_France'] = df_solaire['Date'] + ' ' + df_solaire['Heure']

df_solaire['Date_Heure_France'] = pd.to_datetime(df_solaire['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_solaire = df_solaire.reset_index(drop=True)
df_solaire = df_solaire.sort_values(by='Date_Heure_France', axis = 0, ascending=True)

df_solaire = df_solaire[['Date_Heure_France', 'Prévision J-1']]
df_solaire.columns =['Date_Heure_France', 'Prévision J-1 Solaire'] 


date_debut = df_solaire['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_solaire['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_solaire = df_solaire.set_index('Date_Heure_France')
here = date_range.isin(df_solaire.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_solaire.loc[idx, 'Prévision J-1 Solaire'] = 0

df_solaire.loc[df_solaire.index.year == 2023]
date_debut = df_solaire.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_solaire), freq='1H')
df_solaire.index = date_range

df_solaire.index = df_solaire.index.rename('Date UTC')

df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True

df_solaire.loc[df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True] = 0
df_solaire['Prévision J-1 Solaire'] = df_solaire['Prévision J-1 Solaire'].astype(float)

df_solaire.sample(24)

,Prévision J-1 Solaire
Date UTC,
2017-10-23 16:00:00,1403.76
2017-10-31 11:00:00,2499.91
2017-03-07 15:00:00,1624.60
2019-10-08 19:00:00,0.00
2021-11-03 19:00:00,0.00
2021-07-28 08:00:00,991.67
2023-01-26 04:00:00,0.00
2018-06-24 04:00:00,0.00
2018-06-20 14:00:00,0.00


In [158]:

df_eolien = pd.concat(df_list)

#df_eolien = df_eolien.reset_index(inplace=False).drop('index', axis=1)

mask = df_eolien['Heure'].isna()
df_eolien=df_eolien.loc[~mask]

df_eolien['Heure'] = df_eolien['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_eolien['Date_Heure_France'] = df_eolien['Date'] + ' ' + df_eolien['Heure']

df_eolien['Date_Heure_France'] = pd.to_datetime(df_eolien['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_eolien = df_eolien.reset_index(drop=True)
df_eolien = df_eolien.sort_values(by='Date_Heure_France', axis = 0, ascending=True)


df_eolien = df_eolien[['Date_Heure_France', 'Prévision J-1']]
df_eolien.columns = ['Date_Heure_France', 'Prévision J-1 Eolien']

date_debut = df_eolien['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_eolien['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_eolien = df_eolien.set_index('Date_Heure_France')
here = date_range.isin(df_eolien.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_eolien.loc[idx, 'Prévision J-1 Eolien'] = 0

df_eolien.loc[df_eolien.index.year == 2023]
date_debut = df_eolien.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_eolien), freq='1H')
df_eolien.index = date_range

df_eolien.index = df_eolien.index.rename('Date UTC')

df_eolien['Prévision J-1 Eolien'].str.startswith('-') == True

df_eolien.loc[df_eolien['Prévision J-1 Eolien'].str.startswith('-') == True] = 0
df_eolien['Prévision J-1 Eolien'] = df_eolien['Prévision J-1 Eolien'].astype(float)

df_eolien.sample(24)

,Prévision J-1 Eolien
Date UTC,
2022-06-13 12:00:00,6288.97
2021-05-14 19:00:00,2087.62
2019-11-29 00:00:00,0.00
2019-10-03 11:00:00,3926.06
2017-10-12 03:00:00,0.00
2022-09-08 14:00:00,9406.82
2020-02-08 06:00:00,0.00
2017-09-25 09:00:00,1341.64
2017-09-25 19:00:00,0.05


In [164]:
df_renew = df_solaire.join(df_eolien)
df_renew.to_csv('/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/RENEW_FORECAST_FR.csv')